# 🎭 Simple Face Swapping for Google Colab

**Image-to-image face swapping** - No webcam needed!

📦 **Repository**: [SmashCodeJJ/CIS5810_FinalProject](https://github.com/SmashCodeJJ/CIS5810_FinalProject)  
🔧 **Branch**: `Youxin`

---

## ⚠️ Setup Process

1. ✅ **Enable GPU**: Runtime → Change runtime type → GPU (T4)
2. ✅ **Run Installation Cell** → Installs dependencies
3. 🔄 **Restart Runtime** → Click "Runtime → Restart runtime"
4. ▶️ **Run Face Swap** → Process your images


## Step 1: Installation (Run once, then RESTART RUNTIME)


In [ ]:
# Clone repository
!git clone -b Youxin https://github.com/SmashCodeJJ/CIS5810_FinalProject.git sber-swap
%cd sber-swap

# Install dependencies
%pip install -q -r requirements.txt

# Download models (if needed)
import os
if not os.path.exists('weights/G_unet_2blocks.pth'):
    print("Downloading models...")
    !bash download_models.sh 2>/dev/null || echo "Models may need manual download"

print("\n" + "="*50)
print("✅ Installation complete!")
print("="*50)
print("⚠️  IMPORTANT: Go to Runtime → Restart runtime")
print("    Then skip this cell and run the cells below.")
print("="*50)


## Step 2: Verify Installation (Run after restart)


In [ ]:
%cd /content/sber-swap

import torch
import numpy as np
import cv2

print("="*50)
print("✅ Environment Check")
print("="*50)
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print("🚀 GPU acceleration enabled!")
print("="*50)


## Step 3: Run Face Swap (Image to Image)


In [ ]:
%cd /content/sber-swap

# Run face swap with example images
!python inference.py \
  --image_to_image True \
  --target_image examples/images/beckham.jpg \
  --source_paths examples/images/mark.jpg \
  --out_image_name examples/results/ghost_result.png

print("✅ Face swap complete!")


## Step 4: Display Results


In [ ]:
from IPython.display import display, Image as IPImage
import matplotlib.pyplot as plt
import cv2
import os
import filecmp

# Paths
result_path = '/content/sber-swap/examples/results/ghost_result.png'
target_path = '/content/sber-swap/examples/images/beckham.jpg'
source_path = '/content/sber-swap/examples/images/mark.jpg'

# Check if result exists
if not os.path.exists(result_path):
    print("❌ Result file not found. Check for errors above.")
else:
    # Display comparison
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Source
    if os.path.exists(source_path):
        source = cv2.imread(source_path)
        axes[0].imshow(cv2.cvtColor(source, cv2.COLOR_BGR2RGB))
        axes[0].set_title('Source (Mark)', fontsize=14)
        axes[0].axis('off')
    else:
        axes[0].text(0.5, 0.5, 'Source not found', ha='center')
        axes[0].axis('off')

    # Target
    if os.path.exists(target_path):
        target = cv2.imread(target_path)
        axes[1].imshow(cv2.cvtColor(target, cv2.COLOR_BGR2RGB))
        axes[1].set_title('Target (Beckham)', fontsize=14)
        axes[1].axis('off')
    else:
        axes[1].text(0.5, 0.5, 'Target not found', ha='center')
        axes[1].axis('off')

    # Result
    result = cv2.imread(result_path)
    axes[2].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    axes[2].set_title('RESULT - Face Swapped', fontsize=14, color='green', weight='bold')
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()

    # Check if files are the same
    if os.path.exists(target_path):
        same = filecmp.cmp(target_path, result_path)
        if same:
            print("❌ RESULT IS IDENTICAL TO TARGET - Face swap didn't work")
            print("   Check that both images have clear faces.")
        else:
            print("✅ RESULT IS DIFFERENT - Face swap appears to have worked!")
    else:
        print("⚠️  Could not compare files")


In [ ]:
from google.colab import files
from IPython.display import display, Image as IPImage

# Create directory
!mkdir -p /content/sber-swap/examples/my_images

# Upload target image
print("Upload TARGET image (face to replace):")
uploaded = files.upload()
target_file = None
for filename in uploaded.keys():
    target_file = f"/content/sber-swap/examples/my_images/target_{filename}"
    !mv "{filename}" "{target_file}"
    print(f"✅ Saved: {target_file}")
    display(IPImage(target_file))
    break

# Upload source image
print("\nUpload SOURCE image (face to use):")
uploaded = files.upload()
source_file = None
for filename in uploaded.keys():
    source_file = f"/content/sber-swap/examples/my_images/source_{filename}"
    !mv "{filename}" "{source_file}"
    print(f"✅ Saved: {source_file}")
    display(IPImage(source_file))
    break

# Run face swap
if target_file and source_file:
    print("\n🔄 Processing face swap...")
    !python inference.py \
      --image_to_image True \
      --target_image {target_file} \
      --source_paths {source_file} \
      --out_image_name /content/sber-swap/examples/my_images/result.png
    
    # Display result
    if os.path.exists('/content/sber-swap/examples/my_images/result.png'):
        print("\n✅ Result:")
        display(IPImage('/content/sber-swap/examples/my_images/result.png'))
        files.download('/content/sber-swap/examples/my_images/result.png')
    else:
        print("❌ Result not found. Check for errors above.")
else:
    print("⚠️  Please upload both images.")
